In [1]:
from pulp import *
import pandas as pd
import numpy as np

In [2]:
A5 = LpProblem("hornby",LpMinimize)

In [3]:
df_cost = pd.read_excel("Exhibits.xlsx",sheet_name="cost")
df_demand = pd.read_excel("Exhibits.xlsx",sheet_name="demand")
df_dis_cost = pd.read_excel("Exhibits.xlsx",sheet_name="dis_cost")
df_dis_cost.set_index("Unnamed: 0",inplace=True)

In [4]:
from_cities = df_cost.Location.tolist()#8
to_cities = df_demand.Location.tolist()#22

fixed_cost = df_cost["Fixed cost"].tolist()
fixed_cost = dict(zip(from_cities,fixed_cost))

var_cost = df_cost["Variable cost"].tolist()
var_cost = dict(zip(from_cities,var_cost))

Min_A = df_cost["Min_A"].tolist()
Min_A = dict(zip(from_cities,Min_A))

Max_A = df_cost["Max_A"].tolist()
Max_A = dict(zip(from_cities,Max_A))

dis_cost = makeDict([from_cities,to_cities],\
                    df_dis_cost.values.transpose().tolist())

In [5]:
demand = df_demand.Volume.tolist()
demand = dict(zip(to_cities,demand))
# Decision Variable
open_or_not = LpVariable.dicts("x",from_cities,lowBound=0,cat="Binary")
product = LpVariable.dicts("p",(from_cities,to_cities), lowBound =0, cat="Continuous")

In [6]:
#zero_cost = LpVariable.dicts("z",from_cities,lowBound=0,cat="Binary")

In [7]:
A5 += lpSum([dis_cost[i][j]*product[i][j] for i in from_cities for j in to_cities])+\
      lpSum([fixed_cost[i]*open_or_not[i] for i in from_cities])+\
      lpSum([product[i][j]*var_cost[i] for j in to_cities for i in from_cities])

In [8]:
#Constraints
# products should be smaller than Max_A
for i in from_cities:
    A5 += lpSum([product[i][j] for j in to_cities]) <= open_or_not[i]*Max_A[i]
    
# Products should satisfy the demand    
for j in to_cities:
    A5 += lpSum([product[i][j] for i in from_cities])>=demand[j]
    
# for i in from_cities:
#     bool_zero = sum([product[i][j] for j in to_cities]) >= open_or_not[i]*Min_A[i]
#     A5 += lpSum(zero_cost[i]==int(bool_zero))

In [9]:
A5.writeLP("HornBy.lp")
A5.solve()
print("Status:",LpStatus[A5.status])
print(value(A5.objective))

Status: Optimal
61475.0


In [10]:
output=[]
for i in from_cities:# 8 rows
    var_output=[]
    for j in to_cities: # 22 columns
        var_output.append(product[i][j].varValue)
    var_output.append(open_or_not[i].varValue)
    output.append(var_output)
to_cities.append('Rent')
output_df = pd.DataFrame(output,index=from_cities, columns=to_cities)
output_df

,Atlanta,Birmingham,Buffalo,Charteston,Charlotte,Chattanooga,Chicago,Cincinnati,Cleveland,Columbus,...,Indianapolis,Knoxville,Louisville,Memphis,Nashville,Peoria,Pittsburgh,Richmond,St. Louis,Rent
Atlanta,275.0,160.0,0.0,0.0,135.0,160.0,0.0,0.0,0.0,0.0,...,0.0,125.0,340.0,100.0,210.0,0.0,0.0,0.0,0.0,1.0
Buffalo,0.0,0.0,240.0,0.0,0.0,0.0,0.0,0.0,320.0,220.0,...,0.0,0.0,0.0,0.0,0.0,0.0,280.0,0.0,0.0,1.0
Chicago,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cincinnati,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Detroit,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pittsburgh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Richmond,0.0,0.0,0.0,260.0,0.0,0.0,0.0,200.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,60.0,300.0,0.0,1.0
St. Louis,0.0,0.0,0.0,0.0,0.0,0.0,400.0,0.0,0.0,0.0,...,310.0,0.0,0.0,140.0,0.0,150.0,0.0,0.0,260.0,1.0
